<a href="https://colab.research.google.com/github/JacobaCarr26/comp215/blob/main/lab02-review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2
----------------
#### Name: Jacoba Carr
#### Date: 2025-01-15

By the end of this lab you should be able to:
  * use strings, tuples, lists, and dictionaries (review)
  * use *f-string* to simplify string formatting operations
  * write functions and simple unit tests (review)
  * use *list comprehension* to implement map and filter algorithms


During this lab, you will be introduced to the following:
  * API Keys
  * JSON data formats
  * the `datetime` module to represent dates and times
  * the `pprint` module to format data structures

## Near Earth Objects

In this lab, we'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/) using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS).  But, as usual, let's do the imports first.


In [ ]:
! pip install python-dotenv
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

### dates & f-strings

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [ ]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

2025-01-15 <class 'datetime.date'>
Today is: 2025-01-15


### Make an API Query

Let's get some data from the NEO database; here's a query that gets the observation "feed" for today.  Notice the first line of the block is an 'API key'.  An API key is a unique identifier that authenticates a user when making a request to an API.  Run the block below using the API key 'DEMO_KEY' (just to be sure it works), then go to https://api.nasa.gov and get your own API key.  Use your own API key for the remainder of the lab.

In [ ]:
from dotenv import load_dotenv # to keep API key safe
import os

load_dotenv('API_KEY.env') # load .env file
API_KEY = os.getenv('API_KEY')

if API_KEY:
  print('API_KEY loaded successfully')
else:
  print('API_KEY not found')

# get today's date (as a string)
today = str(datetime.date.today())

# use an f-string to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

# make the API request (recall from lab 1)
response = requests.request("GET", url, headers={}, data={})

# convert the response to json format (recall from lab 1)
data = json.loads(response.text)

# pprint(data)
# here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
#pprint(neos[:1])

API_KEY loaded successfully
12 Near Earth Objects found for 2025-01-15


Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [ ]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

0 potentially hazardous asteroids identified.


### Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.  We can use this `id` to fetch complete orbital and close approach data for the NEO.  For example, this query fetches the complete data set for the first hazardous asteroid...  Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".


In [ ]:
if hazards: # because 2025-01-15 didn't have any
  id = hazards[0]['id']
  url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
  response = requests.request("GET", url, headers={}, data={})
  data = json.loads(response.text)
  #pprint(data)
else:
  print('No hazardous asteroids found')

No hazardous asteroids found


## Exercise 1

In the code cell below, **re-write the accumulator loop** that creates the list of hazards as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [ ]:
# Ex. 1 your code here
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]



## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [ ]:
# Ex. 2 your code here
def closest_approach(close_approach_data):
  ''' Returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list
  (where "miss km" is the miss distance in km).

  Param: "close approach data" is a list of dictionaries with a "close_approach_date",
  "orbiting_body", and 'miss_distance' field.
  '''
  closest = close_approach_data[0]
  for item in close_approach_data:
    if item['orbiting_body'] != 'Earth':
      continue
    if item['miss_distance']['kilometers'] < closest['miss_distance']['kilometers']:
      closest = item
  return (closest['close_approach_date'], closest['miss_distance']['kilometers'])

# Unit test:

url = f'https://api.nasa.gov/neo/rest/v1/neo/3542519?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)
# Access just the close approach data
ca_data_all = data['close_approach_data']

# Filter for specific dates
start_date = datetime.datetime.strptime("2000-08-28", "%Y-%m-%d")
end_date = datetime.datetime.strptime("2005-05-03", "%Y-%m-%d")

ca_data = [
    item for item in ca_data_all
    if start_date <= datetime.datetime.strptime(item["close_approach_date"], "%Y-%m-%d") <= end_date
]

assert closest_approach(ca_data) == ('2001-08-06', '18937090.024000869')

## Challenge Exercise

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [ ]:
from datetime import datetime, timedelta

def closest_approach_triple(data):
  """
  Returns a triple with the (id, date, miss km) of the closest approach to Earth in the list
  (where "miss km" is the miss distance in km).

  Param: "data" is a list of dictionaries with fields "id" and "close approach data" dictionary
  containing "close_approach_date", "orbiting_body", and 'miss_distance' fields.
  """
  # Generate a list of valid close approaches
  valid_approaches = [
    (item['id'], approach['close_approach_date'], float(approach['miss_distance']['kilometers']))
    for item in data
    for approach in item.get('close_approach_data', [])
    if approach.get('orbiting_body') == 'Earth'
  ]

  # Find the closest approach by miss distance
  if valid_approaches:
    return min(valid_approaches, key=lambda x: x[2])  # x[2] is used to sort by miss distance (in km)
  return None

def main():
  """
  Creates a list of NEO's for this week
  Prints the closest approach NEO's id, date, and miss distance.
  """
  today = datetime.now()
  one_week_ago = today - timedelta(weeks=1)

  # Format today and one week ago to start and end
  start = one_week_ago.strftime("%Y-%m-%d")
  end = today.strftime("%Y-%m-%d")

  # Fetch data for the given date range & catch errors
  url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start}&end_date={end}&api_key={API_KEY}'
  response = requests.get(url)
  if response.status_code != 200:
    print(f"Failed to fetch data: {response.status_code}, {response.text}")
    return

  data = json.loads(response.text)

  # Extract list of all NEOs from the date range
  neos = [neo for neo_list in data.get('near_earth_objects', {}).values() for neo in neo_list]

  # Find and print the closest approach & catch events with no close approaches to Earth
  closest = closest_approach_triple(neos)
  if closest:
    print(f"Closest approach: ID={closest[0]}, Date={closest[1]}, Miss Distance={closest[2]} km")
  else:
    print("No close approaches to Earth found in the provided data.")

main()

Closest approach: ID=54516006, Date=2025-01-11, Miss Distance=228199.88205114 km
